## 0. Подготовительная часть ##

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import time
import os
import math
import multiprocessing as mp
import numba
import numpy as np

from SALib.sample import saltelli
from SALib.analyze import sobol

In [3]:
def calc_func(x):
    n = len(x)
    result = 1
    for i in range(n):
        result *= math.exp(((i + 1) % 2) *math.sin(x[i]) ** i + (i % 2) * math.cos(x[i]) ** i)
    return result

In [4]:
problem = {
    'num_vars': 4,
    'names': ['x1', 'x2', 'x3', 'x4'],
    'bounds': [
        [-math.pi, math.pi],
        [-math.pi, math.pi],
        [-math.pi, math.pi],
        [-math.pi, math.pi]
    ]
}

In [5]:
NSAMPLES = 2 ** 19
NSAMPLES

524288

In [6]:
NPROCS = 4

## 1. Анализ чувствительности, тайминги ##

In [8]:
def calc_sensitivity(problem, func, NSAMPLES):

    start_time0 = time.time()
    samples = saltelli.sample(problem, NSAMPLES)
    sampling_time = time.time() - start_time0
    
    y = np.zeros([samples.shape[0]])
    start_time1 = time.time()
    for i, x in enumerate(samples):
        y[i] = func(x)
    calc_time = time.time() - start_time1
    
    start_time2 = time.time()
    sobol_result = sobol.analyze(problem, y)
    sobol_time = time.time() - start_time2
    total_time = time.time() - start_time0
    
    print(f"TOTAL TIME                {total_time} s")
    print(f"     SAMPLING tooks       {sampling_time} s")
    print(f"     CALCULATION tooks    {calc_time} s")
    print(f"     SOBOL ANALYSIS tooks {sobol_time} s")
    print()
    print("       S1 = ", sobol_result['S1'])

    return total_time

In [9]:
print("=============== Straight forward calculation ===============")
straight_forward_calc_time = calc_sensitivity(problem, calc_func, NSAMPLES)

=============== Straight forward calculation ===============
TOTAL TIME                128.50101685523987 s
     SAMPLING tooks       16.82408881187439 s
     CALCULATION tooks    16.95706605911255 s
     SOBOL ANALYSIS tooks 94.71982836723328 s

       S1 =  [0.         0.38373294 0.10847947 0.28994713]


## 2. Numba ##

In [10]:
@numba.njit
def calc_func_nb(x):
    n = len(x)
    result = 1
    for i in range(n):
        result *= math.exp(((i + 1) % 2) * math.sin(x[i]) ** i + (i % 2) * math.cos(x[i]) ** i)
    return result

In [11]:
print("=============== Numba calculation ===============")
numba_calc_time = calc_sensitivity(problem, calc_func_nb, NSAMPLES)

=============== Numba calculation ===============
TOTAL TIME                113.1530351638794 s
     SAMPLING tooks       16.902222156524658 s
     CALCULATION tooks    3.438960075378418 s
     SOBOL ANALYSIS tooks 92.80640006065369 s

       S1 =  [0.         0.38373294 0.10847947 0.28994713]


## 3.  Multiprocessing ##

In [7]:
def calc_sensitivity_mp(problem, func, NSAMPLES):

    start_time0 = time.time()
    samples = saltelli.sample(problem, NSAMPLES)
    sampling_time = time.time() - start_time0
    
    y = np.zeros([samples.shape[0]])
    start_time1 = time.time()
    with mp.Pool(NPROCS) as pool:
        y = pool.map(func, samples.tolist())
    calc_time = time.time() - start_time1
    
    total_time = time.time() - start_time0
    
    print(f"TOTAL TIME                {total_time} s")
    print(f"     SAMPLING tooks       {sampling_time} s")
    print(f"     CALCULATION tooks    {calc_time} s")

    return total_time

In [ ]:
print("=============== Multiprocessing calculation ===============")
mp_calc_time = calc_sensitivity_mp(problem, calc_func, NSAMPLES)

=============== Multiprocessing calculation ===============
